In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append('..')
from bundle import fpgapy as fp
from pyclk import Trace

%matplotlib inline

In [ ]:
fp.fpga.config

In [ ]:
fpga = fp.fpga
#fpga.set_cycle_nb(100)

trace = Trace()
for i in range(fp.fpga.config['mem_nb']):
    trace.add(fpga.u_mem[i].i_addr)
    trace.add(fpga.u_mem[i].o_dout)
    trace.add(fpga.u_mem[i].i_din)
    trace.add(fpga.u_mem[i].i_wena)
for i in range(fp.fpga.config['iter_nb']):
    trace.add(fpga.u_iter[i].o_wena)
#    trace.add(fpga.u_iter[i].o_done)
#    trace.add(fpga.u_iter[i].i_ack)
#    trace.add(fpga.u_iter[i].i_data_nb)
fpga.set_trace(trace)

In [ ]:
size = 5000
a0 = fp.arange(size)
res = (a0 + a0) * a0
print(res[:100])

In [ ]:
a0_true = np.arange(size)
res_true = (a0_true + a0_true) * a0_true
print(res_true[:100])

In [ ]:
if np.array_equal(res, res_true):
    print('Success!')
else:
    print('Failed!')

In [ ]:
res = np.array(res)
diff_idx = np.where(res != res_true)
plt.figure(figsize=(15, 5))
plt.scatter(np.arange(res_true.size), res_true, c='b')
plt.scatter(diff_idx, res[diff_idx], c='r', label='FPGA')
plt.scatter(diff_idx, res_true[diff_idx], c='g', label='NumPy')
#plt.scatter(diff_idx, z_true[diff_idx]-z[diff_idx], c='r')
plt.xlim(0, size)
plt.ylim(1, size+1)
plt.legend(loc='upper left')
plt.title('Differences')
plt.show()

In [ ]:
res[diff_idx]

In [ ]:
res_true[diff_idx]

In [ ]:
trace.plot()